# Efficient Exploration through ALPaCA Q Learning
Course project for CS332: Advanced Survey on Reinforcement Learning

Fengjun Yang, 2018

In [52]:
%reload_ext autoreload
%autoreload 2

# ALPaCA
from agent.alpaca import ALPaCA
from agent.alpacaQ import ALPaCAQ, EGreedyALPaCAQ, TS_ALPaCAQ, TS_MC_ALPaCAQ
from agent.alpacaoffline import ALPaCAOffline

# Mountain Car related
from metamountaincar.mmcenv import MetaMountainCar as MMC
from metamountaincar.utility import *
from metamountaincar.VI import mmcVI
from metamountaincar.mmcDataset import mmcDataset, mmcOfflineDataset
from metamountaincar.mmcNormalize import mmcNorm

# Utility
import numpy as np
import yaml
from tqdm import tqdm

In [2]:
# Load configuration file
cfg_filename = 'config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

## Generate value functions for offline training. 
Technically, one can use mmcDataset directly for ALPaCA offline training. Here, however, we store and reuse sampled data.

SKIP IF USING PRESAMPLED DATA

In [ ]:
# Sample Q functions and store in DATA_DIR
for i in tqdm(range(N_DATASET)):
    sample = mmcDataset(config).sample(N_FUNCS, N_SAMPLES)
    with open(DATA_DIR+str(i)+'.yml', 'w') as yml_file:
        yml_file.write(yaml.dump(sample))

## Offline training

In [3]:
# Load dataset for offline training
dataset = mmcOfflineDataset(config)

# Initialize offline alpaca agent
offline = ALPaCAOffline(config)

In [3]:
# Initialize offline alpaca agent
offline = ALPaCAOffline(config)

In [ ]:
# Offline training
offline.train_offline(dataset, 5000)

In [23]:
# Save agent in a specified directory
# offline.save_agent('./saveagent/')

Saved to: ./saveagent/


In [4]:
offline.load_agent('./saveagent/')

INFO:tensorflow:Restoring parameters from ./saveagent/
Restored model from: ./saveagent/


## Mountain Car Experiment / RL

In [6]:
env = MMC(config)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [61]:
eg_agent = EGreedyALPaCAQ(offline, env)
ts_agent = TS_ALPaCAQ(offline, env)

In [63]:
res = learning_rate(ts_agent, env, num_episodes=5, num_test_plays=1)

100%|██████████| 10/10 [00:22<00:00,  2.28s/it]


## Clean up

In [ ]:
env.close()

In [65]:
offline.sess.close()